## Classification using SparkML


## __Table of Contents__

<ol>
  <li>
    <a href="#Objectives">Objectives
    </a>
  </li>
  <li>
    <a href="#Datasets">Datasets
    </a>
  </li>
  <li>
    <a href="#Setup">Setup
    </a>
    <ol>
      <li>
        <a href="#Installing-Required-Libraries">Installing Required Libraries
        </a>
      </li>
      <li>
        <a href="#Importing-Required-Libraries">Importing Required Libraries
        </a>
      </li>
    </ol>
  </li>
  <li>
    <a href="#Examples">Examples
    </a>
    <ol>
    <li>
      <a href="#Task-1---Create-a-spark-session">Task 1 - Create a spark session
      </a>
    </li>
    <li>
      <a href="#Task-2---Load-the-data-in-a-csv-file-into-a-dataframe">Task 2 - Load the data in a csv file into a dataframe
      </a>
    </li>
    <li>
      <a href="#Task-3---Identify-the-label-column-and-the-input-columns">Task 3 - Identify the label column and the input columns
      </a>
    </li>
    <li>
      <a href="#Task-4---Split-the-data">Task 4 - Split the data
      </a>
    </li>
    <li>
      <a href="#Task-5---Build-and-Train-a-Logistic-Regression-Model">Task 5 - Build and Train a Logistic Regression Model
      </a>
    </li>
    <li>
      <a href="#Task-6---Evaluate-the-model">Task 6 - Evaluate the model
      </a>
    </li>
    </ol>
  </li>
  <li>
    <a href="#Exercises">Exercises
    </a>
  </li>
  <ol>
    <li>
      <a href="#Exercise-1---Create-a-spark-session">Exercise 1 - Create a spark session
      </a>
    </li>
    <li>
      <a href="#Exercise-2---Load-the-data-in-a-csv-file-into-a-dataframe">Exercise 2 - Load the data in a csv file into a dataframe
      </a>
    </li>
    <li>
      <a href="#Exercise-3---Identify-the-label-column-and-the-input-columns">Exercise 3 - Identify the label column and the input columns
      </a>
    </li>
    <li>
      <a href="#Exercise-4---Split-the-data">Exercise 4 - Split the data
      </a>
    </li>
    <li>
      <a href="#Exercise-5---Build-and-Train-a-Logistic-Regression-Model">Exercise 5 - Build and Train a Logistic Regression Model
      </a>
    </li>
    <li>
      <a href="#Exercise-6---Evaluate-the-model">Exercise 6 - Evaluate the model
      </a>
    </li>
  </ol>
</ol>


## Objectives

After completing this lab you will be able to:

 - Use PySpark to connect to a spark cluster.
 - Create a spark session.
 - Read a csv file into a data frame.
 - Split the dataset into training and testing sets.
 - Use VectorAssembler to combine multiple columns into a single vector column
 - Use Logistic Regression to build a classification model.
 - Use metrics to evaluate the model.
 - Stop the spark session
 
 
 


## Datasets

In this lab you will be using dataset(s):

 - Dry Bean dataset. Available at https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset 
 - Modified version of iris dataset. Available at https://archive.ics.uci.edu/ml/datasets/Iris 
 


----


## Setup


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

from pyspark.sql import SparkSession

#import functions/Classes for sparkml

from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# import functions/Classes for metrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Examples


## Task 1 - Create a spark session


In [3]:
#Create SparkSession
#Ignore any warnings by SparkSession command

spark = SparkSession.builder.appName("Classification using SparkML").getOrCreate()

24/06/17 18:33:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 18:34:00 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Task 2 - Load the data in a csv file into a dataframe


Download the data file


In [4]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/drybeans.csv


--2024-06-17 18:34:04--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/drybeans.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2484759 (2.4M) [text/csv]
Saving to: ‘drybeans.csv’

drybeans.csv        100%[===================>]   2.37M  --.-KB/s    in 0.06s   

2024-06-17 18:34:04 (39.2 MB/s) - ‘drybeans.csv’ saved [2484759/2484759]



Load the dataset into the spark dataframe


In [5]:
# using the spark.read.csv function we load the data into a dataframe.
# the header = True mentions that there is a header row in out csv file
# the inferSchema = True, tells spark to automatically find out the data types of the columns.

# Load mpg dataset
beans_data = spark.read.csv("drybeans.csv", header=True, inferSchema=True)


Print the schema of the dataset


In [6]:
beans_data.printSchema()

root
 |-- Area: integer (nullable = true)
 |-- Perimeter: double (nullable = true)
 |-- MajorAxisLength: double (nullable = true)
 |-- MinorAxisLength: double (nullable = true)
 |-- AspectRation: double (nullable = true)
 |-- Eccentricity: double (nullable = true)
 |-- ConvexArea: integer (nullable = true)
 |-- EquivDiameter: double (nullable = true)
 |-- Extent: double (nullable = true)
 |-- Solidity: double (nullable = true)
 |-- roundness: double (nullable = true)
 |-- Compactness: double (nullable = true)
 |-- ShapeFactor1: double (nullable = true)
 |-- ShapeFactor2: double (nullable = true)
 |-- ShapeFactor3: double (nullable = true)
 |-- ShapeFactor4: double (nullable = true)
 |-- Class: string (nullable = true)



In [7]:
beans_data.show(5)

+-----+---------+---------------+---------------+------------+------------+----------+-------------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----+
| Area|Perimeter|MajorAxisLength|MinorAxisLength|AspectRation|Eccentricity|ConvexArea|EquivDiameter|     Extent|   Solidity|  roundness|Compactness|ShapeFactor1|ShapeFactor2|ShapeFactor3|ShapeFactor4|Class|
+-----+---------+---------------+---------------+------------+------------+----------+-------------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----+
|28395|  610.291|    208.1781167|     173.888747| 1.197191424| 0.549812187|     28715|  190.1410973|0.763922518|0.988855999|0.958027126|0.913357755| 0.007331506| 0.003147289| 0.834222388| 0.998723889|SEKER|
|28734|  638.018|    200.5247957|    182.7344194| 1.097356461| 0.411785251|     29172|  191.2727505|0.783968133|0.984985603|0.887033637|0.953860842| 0.006978659| 0.00356362

beans_dataPrint top 5 rows of selected columns from the dataset


In [8]:
beans_data.select(["Area","Perimeter","Solidity","roundness","Compactness","Class"]).show(5)

+-----+---------+-----------+-----------+-----------+-----+
| Area|Perimeter|   Solidity|  roundness|Compactness|Class|
+-----+---------+-----------+-----------+-----------+-----+
|28395|  610.291|0.988855999|0.958027126|0.913357755|SEKER|
|28734|  638.018|0.984985603|0.887033637|0.953860842|SEKER|
|29380|   624.11|0.989558774|0.947849473|0.908774239|SEKER|
|30008|  645.884|0.976695743|0.903936374|0.928328835|SEKER|
|30140|  620.134| 0.99089325|0.984877069|0.970515523|SEKER|
+-----+---------+-----------+-----------+-----------+-----+
only showing top 5 rows



Print the value counts for the column 'Class'


In [9]:
beans_data.groupBy('Class').count().orderBy('count').show()

+--------+-----+
|   Class|count|
+--------+-----+
|  BOMBAY|  522|
|BARBUNYA| 1322|
|    CALI| 1630|
|   HOROZ| 1928|
|   SEKER| 2027|
|    SIRA| 2636|
|DERMASON| 3546|
+--------+-----+



Convert the string column "Class" into a numberic column named "Label"


In [10]:
# Convert Class column from string to numerical values
indexer = StringIndexer(inputCol="Class", outputCol="label")
beans_data = indexer.fit(beans_data).transform(beans_data)


Print the value counts for the column 'label'


In [11]:
beans_data.groupBy('label').count().orderBy('count').show()

[Stage 9:===============================================>       (172 + 8) / 200]

+-----+-----+
|label|count|
+-----+-----+
|  6.0|  522|
|  5.0| 1322|
|  4.0| 1630|
|  3.0| 1928|
|  2.0| 2027|
|  1.0| 2636|
|  0.0| 3546|
+-----+-----+



## Task 3 - Identify the label column and the input columns


We ask the VectorAssembler to group a bunch of inputCols as single column named "features"


Use "Area","Perimeter","Solidity","roundness","Compactness" as input columns


In [12]:
# Prepare feature vector
assembler = VectorAssembler(inputCols=["Area","Perimeter","Solidity","roundness","Compactness"], outputCol="features")
beans_transformed_data = assembler.transform(beans_data)


Display the assembled "features" and the label column "MPG"


In [13]:
beans_transformed_data.select("features","label").show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[28395.0,610.291,...|  2.0|
|[28734.0,638.018,...|  2.0|
|[29380.0,624.11,0...|  2.0|
|[30008.0,645.884,...|  2.0|
|[30140.0,620.134,...|  2.0|
|[30279.0,634.927,...|  2.0|
|[30477.0,670.033,...|  2.0|
|[30519.0,629.727,...|  2.0|
|[30685.0,635.681,...|  2.0|
|[30834.0,631.934,...|  2.0|
|[30917.0,640.765,...|  2.0|
|[31091.0,638.558,...|  2.0|
|[31107.0,640.594,...|  2.0|
|[31158.0,642.626,...|  2.0|
|[31158.0,641.105,...|  2.0|
|[31178.0,636.888,...|  2.0|
|[31202.0,644.454,...|  2.0|
|[31203.0,639.782,...|  2.0|
|[31272.0,638.666,...|  2.0|
|[31335.0,635.011,...|  2.0|
+--------------------+-----+
only showing top 20 rows



## Task 4 - Split the data


We split the data set in the ratio of 70:30. 70% training data, 30% testing data.


In [14]:
# Split data into training and testing sets
(training_data, testing_data) = beans_transformed_data.randomSplit([0.7, 0.3], seed=42)


The random_state variable "seed" controls the shuffling applied to the data before applying the split. Pass the same integer for reproducible output across multiple function calls


## Task 5 - Build and Train a Logistic Regression Model


Create a LR model and train the model using the training data set


In [15]:
# Ignore any warnings

lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(training_data)

24/06/17 19:01:43 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/06/17 19:01:43 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


## Task 6 - Evaluate the model


Your model is now trained. We use the testing data to make predictions.


In [16]:
# Make predictions on testing data
predictions = model.transform(testing_data)

##### Accuracy


In [18]:
# Evaluate model performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy =", accuracy)


Accuracy = 0.8937376725838264


##### Precision


In [19]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator.evaluate(predictions)
print("Precision =", precision)


Precision = 0.8934405575280979


##### Recall


In [20]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator.evaluate(predictions)
print("Recall =", recall)


Recall = 0.8937376725838265


##### F1 Score


In [21]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)
print("F1 score = ", f1_score)


F1 score =  0.8933106212109942


Stop Spark Session


In [22]:
spark.stop()

# Iris Flower Classification


In this exercise you will use sparkml to classify the iris flower dataset.


### Exercise 1 - Create a spark session


Create a spark session with appname "Iris Flower Classification"


In [23]:
# Initialize SparkSession
spark = SparkSession.builder.appName("Iris Flower Classification").getOrCreate()


24/06/17 19:03:33 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Exercise 2 - Load the data in a csv file into a dataframe


Download the data set


In [24]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/iris.csv


--2024-06-17 19:04:28--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/iris.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4612 (4.5K) [text/csv]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   4.50K  --.-KB/s    in 0s      

2024-06-17 19:04:28 (37.3 MB/s) - ‘iris.csv’ saved [4612/4612]



Load the dataset into a spark dataframe


In [25]:
iris_data = spark.read.csv("iris.csv", header=True, inferSchema=True)


Display top 5 rows from dataset


In [26]:
iris_data.show(5)

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



### Exercise 3 - Identify the label column and the input columns


Transform the string column "Species" to a column of numerical values named "label"


In [27]:
indexer = StringIndexer(inputCol="Species", outputCol="label" )
iris_data = indexer.fit(iris_data).transform(iris_data)


In [28]:
iris_data.show(5)

+-------------+------------+-------------+------------+-----------+-----+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|
+-------------+------------+-------------+------------+-----------+-----+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|  0.0|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|  0.0|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|  0.0|
+-------------+------------+-------------+------------+-----------+-----+
only showing top 5 rows



In [29]:
iris_data.groupBy('label').count().orderBy('count').show()

[Stage 7:==========================================>           (157 + 17) / 200]

+-----+-----+
|label|count|
+-----+-----+
|  0.0|   50|
|  1.0|   50|
|  2.0|   50|
+-----+-----+



Assemble the columns columns "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm" into a single column named "features"


In [30]:
# Prepare feature vector
assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")
iris_transformed_data = assembler.transform(iris_data)


Print the vectorized features and label columns


In [36]:
iris_transformed_data.select("features","label").show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
+-----------------+-----+
only showing top 5 rows



### Exercise 4 - Split the data


Split the dataset into training and testing sets in the ratio of 70:30. Use 42 as seed.


In [34]:

(training_data, testing_data) = iris_transformed_data.randomSplit([0.7,0.3], seed = 42)


In [35]:
testing_data.show(5)

+-------------+------------+-------------+------------+---------------+-----+-----------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|        Species|label|         features|
+-------------+------------+-------------+------------+---------------+-----+-----------------+
|          4.4|         2.9|          1.4|         0.2|    Iris-setosa|  0.0|[4.4,2.9,1.4,0.2]|
|          4.4|         3.0|          1.3|         0.2|    Iris-setosa|  0.0|[4.4,3.0,1.3,0.2]|
|          4.4|         3.2|          1.3|         0.2|    Iris-setosa|  0.0|[4.4,3.2,1.3,0.2]|
|          4.8|         3.1|          1.6|         0.2|    Iris-setosa|  0.0|[4.8,3.1,1.6,0.2]|
|          4.9|         2.4|          3.3|         1.0|Iris-versicolor|  1.0|[4.9,2.4,3.3,1.0]|
+-------------+------------+-------------+------------+---------------+-----+-----------------+
only showing top 5 rows



### Exercise 5 - Build and Train a Logistic Regression Model


Build a Logistic Regression model and train it


In [37]:

lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(training_data)

Predict the values using the test data


### Exercise 6 - Evaluate the model


Your model is now trained. Make the model predict on testing_data


In [ ]:

predictions = model.transform(testing_data)

Print the metrics :
- Accuracy
- Precision
- Recall
- F1 Score


In [39]:
# Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy =", accuracy)

# Precision
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator.evaluate(predictions)
print("Precision =", precision)

# Recall
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator.evaluate(predictions)
print("Recall =", recall)

# F1
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)
print("F1 =", f1)

Accuracy = 0.9722222222222222
Precision = 0.9743589743589745
Recall = 0.9722222222222221
F1 = 0.9722222222222223
